In [9]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras import datasets


In [10]:
import cv2
import pandas as pd
from keras.utils import to_categorical 

def load_images_and_labels(file_path):
    dataset = pd.read_csv(os.path.join(file_path, "_classes.csv"))
    
    classes = dataset.columns
    file_names = dataset.iloc[:, 0].values
    max_label = dataset.iloc[:, 1:].values
    max_label = np.argmax(max_label, axis=1)

    images = []
    new_labels = []

    for i, img_path in enumerate(file_names):
        full_path = os.path.join(file_path, img_path)
        if not os.path.exists(full_path):
            continue

        img = cv2.imread(full_path)
        if img is None:
            continue

        img = cv2.resize(img, (224, 224))
        img = img / 255.0

        if max_label[i] != 6:
            images.append(img)
            new_labels.append(max_label[i])

    new_labels = to_categorical(new_labels, num_classes=6)

    return np.array(images, dtype='float32'), new_labels

In [11]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model

def AlexNet():
  inp = layers.Input((224, 224, 3))
  x = layers.Conv2D(96, 11, 4, activation='relu')(inp)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Conv2D(256, 5, 1, activation='relu')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Conv2D(384, 3, 1, activation='relu')(x)
  x = layers.Conv2D(384, 3, 1, activation='relu')(x)
  x = layers.Conv2D(256, 3, 1, activation='relu')(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Flatten()(x)
  x = layers.Dense(4096, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(4096, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(6, activation='softmax')(x) 

  model = Model(inputs=inp, outputs=x)

  return model

model = AlexNet()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 batch_normalization_2 (Batc  (None, 54, 54, 96)       384       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 26, 26, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 22, 22, 256)       614656    
                                                                 
 batch_normalization_3 (Batc  (None, 22, 22, 256)      1024

In [16]:
import pandas as pd
import os
import numpy as np

# Load the CSV file into a DataFrame
train_data_dir = 'train/'
validation_data_dir = 'valid/'
test_data_dir = 'test/'  

# Load all images and labels
train_images, train_labels = load_images_and_labels(train_data_dir)
val_images, val_labels = load_images_and_labels(validation_data_dir)
test_images, test_labels = load_images_and_labels(test_data_dir)


In [13]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

model.fit(train_images, train_labels, batch_size=32, epochs=5, validation_data=(val_images, val_labels))

model.save('AlexNet_without_unlabeled.h5')

#T : Jawaban Benar
#Positive / P : Jawaban predict yang nilainya benar
# Accuracy: (TP + TN) / (TP + TN + FP + FN)

# Precision: (TP) / (TP + FP)

# Recall: (TP) / (TP + FN)





Epoch 1/5
192/192 [==============================] - 96s 490ms/step - loss: 1.7631 - accuracy: 0.2275 - val_loss: 2.0595 - val_accuracy: 0.1675
Epoch 2/5
192/192 [==============================] - 97s 503ms/step - loss: 1.6690 - accuracy: 0.3060 - val_loss: 2.0499 - val_accuracy: 0.2074
Epoch 3/5
192/192 [==============================] - 98s 512ms/step - loss: 1.5592 - accuracy: 0.3621 - val_loss: 1.6637 - val_accuracy: 0.3182
Epoch 4/5
192/192 [==============================] - 97s 506ms/step - loss: 1.4629 - accuracy: 0.4099 - val_loss: 2.3108 - val_accuracy: 0.2012
Epoch 5/5
192/192 [==============================] - 100s 519ms/step - loss: 1.3868 - accuracy: 0.4562 - val_loss: 1.6933 - val_accuracy: 0.3227


In [15]:
# To Continue the Training run this cell

from keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Use ReduceLR
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0.00001)

# Load the model
model = load_model('AlexNet_without_unlabeled.h5')

# Continue training
model.fit(train_images, train_labels, batch_size=32, epochs=5, validation_data=(val_images, val_labels), callbacks=reduce_lr)

model.save('AlexNet_without_unlabeled.h5')

Epoch 1/5
192/192 [==============================] - 97s 504ms/step - loss: 1.2866 - accuracy: 0.4948 - val_loss: 1.4957 - val_accuracy: 0.3935 - lr: 1.0000e-04
Epoch 2/5
192/192 [==============================] - 99s 515ms/step - loss: 1.1757 - accuracy: 0.5467 - val_loss: 1.5515 - val_accuracy: 0.4227 - lr: 1.0000e-04
Epoch 3/5
192/192 [==============================] - 98s 511ms/step - loss: 1.0833 - accuracy: 0.5873 - val_loss: 1.6878 - val_accuracy: 0.2962 - lr: 1.0000e-04
Epoch 4/5
192/192 [==============================] - 98s 513ms/step - loss: 0.9409 - accuracy: 0.6425 - val_loss: 1.5359 - val_accuracy: 0.4452 - lr: 1.0000e-04
Epoch 5/5
192/192 [==============================] - 99s 514ms/step - loss: 0.8333 - accuracy: 0.6834 - val_loss: 1.5163 - val_accuracy: 0.4474 - lr: 1.0000e-04


In [17]:
# Test Accuracy

from tensorflow.keras.models import load_model

model = load_model('AlexNet_without_unlabeled.h5')

test_loss, test_accuracy = model.evaluate(val_images, val_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

56/56 [==============================] - 5s 79ms/step - loss: 1.5163 - accuracy: 0.4474
Test Loss: 1.5162811279296875
Test Accuracy: 0.4474423825740814
